# Unit 4 Improving the Model Performance

Welcome back\! In the previous lesson, you learned how to evaluate your CNN model’s performance and visualize its predictions. You also explored important metrics like accuracy, precision, recall, and the F1 score, and saw how to interpret a classification report and confusion matrix. Now that you know how to measure your model’s strengths and weaknesses, it’s time to take the next step: making your model even better.

In this lesson, you will learn practical techniques to improve your CNN’s performance on sketch recognition tasks. These methods are widely used in real-world deep learning projects to help models generalize better and avoid common pitfalls like overfitting. By the end of this lesson, you’ll be able to apply these improvements to your own models and see the difference in results.

### What You'll Learn

You will focus on two powerful techniques for boosting your model’s performance:

  * **Dropout Layers:** Dropout is a simple but effective way to prevent overfitting. It works by randomly “dropping out” (ignoring) a fraction of the neurons during training, which forces the model to learn more robust features. You’ll see how to add a `Dropout` layer to your CNN and understand why it helps.
  * **Early Stopping:** Sometimes, training a model for too many epochs can actually make it perform worse on new data. Early stopping is a technique that monitors your model’s performance on the validation set and automatically stops training when the model stops improving. This saves time and helps you get the best version of your model.

Here’s a quick look at how these improvements appear in code:

```python
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Add a Dropout layer after a Dense layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # 50% of neurons will be dropped during training

# Set up EarlyStopping
early_stopping = EarlyStopping(
    patience=10,
    monitor='val_accuracy',
    restore_best_weights=True)
```

You’ll also notice some other helpful changes in the code, such as using more training data, adding batch normalization, and making the model a bit deeper. These changes help the model learn better and make the training process more stable.

### Why It Matters

Improving your model’s performance is a key part of building reliable AI systems. Overfitting is a common problem where a model does well on training data but fails on new, unseen data. Dropout and early stopping are two of the most effective tools to fight overfitting and make your model more robust.

By mastering these techniques, you’ll be able to:

  * Build models that generalize well to new sketches and drawings.
  * Train faster and avoid wasting time on unnecessary epochs.
  * Understand how to tune your model for the best possible results.

Ready to see these improvements in action? Let’s move on to the practice section and apply these techniques to your own sketch recognition model\!



## Preventing Overfitting with Dropout and EarlyStopping

Great job analyzing those misclassified sketches! Now, let's improve your model to reduce these errors. In this task, you'll add a `Dropout` layer after the Dense layer in your CNN.

Dropout works by randomly disabling neurons during training, which prevents your network from becoming too dependent on any single neuron. This forces the model to learn more robust features and reduces overfitting.

```python
import os
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="Your `PyDataset` class should call", category=UserWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  

import urllib.request
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# Data loading and preprocessing (already done for you)
categories = ['cat', 'house', 'apple']
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

os.makedirs('quickdraw_data', exist_ok=True)

for category in categories:
    file_path = f'quickdraw_data/{category}.npy'
    if not os.path.exists(file_path):
        print(f"Downloading {category}...")
        urllib.request.urlretrieve(base_url + category + '.npy', file_path)
    else:
        print(f"{category}.npy already exists.")

# Load and prepare data
data = []
labels = []
IMAGE_COUNT = 3000

for idx, cat in enumerate(categories):
    filepath = f'quickdraw_data/{cat}.npy'
    imgs = np.load(filepath)[:IMAGE_COUNT]
    if imgs.dtype != np.uint8:
        imgs = imgs.astype(np.uint8)
    data.append(imgs)
    labels.append(np.full(imgs.shape[0], idx))

# Combine all data and labels
data = np.concatenate(data, axis=0)
labels = np.concatenate(labels, axis=0)

# Shuffle data
indices = np.arange(len(data))
np.random.shuffle(indices)
data, labels = data[indices], labels[indices]

# Reshape and normalize
data = data.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(f"Training data shape: {x_train.shape}, Testing data shape: {x_test.shape}")

# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

# Fit the generator to the training data
datagen.fit(x_train)

def build_improved_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28,28,1)),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        # TODO: Add a Dropout layer with a rate of 0.5
        ________,
        tf.keras.layers.Dense(len(categories), activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Build the improved model
model = build_improved_cnn()

# Print model summary
model.summary()

# Train the model with early stopping
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                   epochs=1,
                   validation_data=(x_test, y_test),
                   steps_per_epoch=len(x_train) // 32)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

# Get predictions
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Find misclassified images
misclassified_indices = np.where(y_pred_classes != y_test)[0]
print(f"Found {len(misclassified_indices)} misclassified images")

# Visualize misclassified images
plt.figure(figsize=(10, 10))
for i in range(min(9, len(misclassified_indices))):
    idx = misclassified_indices[i]
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f"True: {categories[y_test[idx]]}\nPred: {categories[y_pred_classes[idx]]}")
    plt.axis('off')
plt.tight_layout()
plt.savefig('static/images/misclassified.png')
print("Misclassified images visualization saved to 'static/images/misclassified.png'")

```

To add a `Dropout` layer with a rate of 0.5, you should replace the `________` placeholder with `tf.keras.layers.Dropout(0.5)`.

Here's the corrected `build_improved_cnn` function:

```python
import os
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="Your `PyDataset` class should call", category=UserWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  

import urllib.request
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# Data loading and preprocessing (already done for you)
categories = ['cat', 'house', 'apple']
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

os.makedirs('quickdraw_data', exist_ok=True)

for category in categories:
    file_path = f'quickdraw_data/{category}.npy'
    if not os.path.exists(file_path):
        print(f"Downloading {category}...")
        urllib.request.urlretrieve(base_url + category + '.npy', file_path)
    else:
        print(f"{category}.npy already exists.")

# Load and prepare data
data = []
labels = []
IMAGE_COUNT = 3000

for idx, cat in enumerate(categories):
    filepath = f'quickdraw_data/{cat}.npy'
    imgs = np.load(filepath)[:IMAGE_COUNT]
    if imgs.dtype != np.uint8:
        imgs = imgs.astype(np.uint8)
    data.append(imgs)
    labels.append(np.full(imgs.shape[0], idx))

# Combine all data and labels
data = np.concatenate(data, axis=0)
labels = np.concatenate(labels, axis=0)

# Shuffle data
indices = np.arange(len(data))
np.random.shuffle(indices)
data, labels = data[indices], labels[indices]

# Reshape and normalize
data = data.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(f"Training data shape: {x_train.shape}, Testing data shape: {x_test.shape}")

# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

# Fit the generator to the training data
datagen.fit(x_train)

def build_improved_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28,28,1)),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5), # Added Dropout layer
        tf.keras.layers.Dense(len(categories), activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Build the improved model
model = build_improved_cnn()

# Print model summary
model.summary()

# Train the model with early stopping
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                   epochs=1,
                   validation_data=(x_test, y_test),
                   steps_per_epoch=len(x_train) // 32)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

# Get predictions
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Find misclassified images
misclassified_indices = np.where(y_pred_classes != y_test)[0]
print(f"Found {len(misclassified_indices)} misclassified images")

# Visualize misclassified images
plt.figure(figsize=(10, 10))
for i in range(min(9, len(misclassified_indices))):
    idx = misclassified_indices[i]
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f"True: {categories[y_test[idx]]}\nPred: {categories[y_pred_classes[idx]]}")
    plt.axis('off')
plt.tight_layout()
plt.savefig('static/images/misclassified.png')
print("Misclassified images visualization saved to 'static/images/misclassified.png'")
```

## Automatic Training Termination with EarlyStopping

Now that you've analyzed your model's performance with detailed metrics, let's implement another key technique to prevent overfitting: Early Stopping.

When training neural networks, we often face a challenge in which training for too many epochs can lead to the model memorizing the training data instead of learning general patterns. This is called overfitting and hurts performance on new data.

Early Stopping automatically monitors your model's performance on validation data and stops training when it detects that performance has stopped improving. This saves training time and helps you capture your model at its best point.

In this task, you'll add an EarlyStopping callback to your sketch recognition model. This will monitor validation accuracy and automatically stop training when the model stops improving for a specified number of epochs.

Note, in this practice EarlyStopping won't make a significant difference because the model is already set to train for only one epoch. We set the number of epochs to one to keep the training time short. However, in a real-world scenario, you would typically train for many epochs (e.g., 50 or more) and use early stopping to prevent overfitting.


```python
import os
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="Your `PyDataset` class should call", category=UserWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import urllib.request
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# Data loading and preprocessing (already done for you)
categories = ['cat', 'house', 'apple']
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

os.makedirs('quickdraw_data', exist_ok=True)

for category in categories:
    file_path = f'quickdraw_data/{category}.npy'
    if not os.path.exists(file_path):
        print(f"Downloading {category}...")
        urllib.request.urlretrieve(base_url + category + '.npy', file_path)
    else:
        print(f"{category}.npy already exists.")

# Load and prepare data
data = []
labels = []
IMAGE_COUNT = 3000

for idx, cat in enumerate(categories):
    filepath = f'quickdraw_data/{cat}.npy'
    imgs = np.load(filepath)[:IMAGE_COUNT]
    if imgs.dtype != np.uint8:
        imgs = imgs.astype(np.uint8)
    data.append(imgs)
    labels.append(np.full(imgs.shape[0], idx))

# Combine all data and labels
data = np.concatenate(data, axis=0)
labels = np.concatenate(labels, axis=0)

# Shuffle data
indices = np.arange(len(data))
np.random.shuffle(indices)
data, labels = data[indices], labels[indices]

# Reshape and normalize
data = data.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(f"Training data shape: {x_train.shape}, Testing data shape: {x_test.shape}")

# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

# Fit the generator to the training data
datagen.fit(x_train)

def build_simple_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28,28,1)),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(categories), activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Build the model
model = build_simple_cnn()

# Print model summary
model.summary()

# TODO: Create an EarlyStopping callback that:
#  - Monitors validation accuracy
#  - Has a patience of 5 epochs
#  - Restores the best weights
#  - Provides verbose output
early_stopping = ________

# Train the model with early stopping
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                   epochs=1,  # We can set a higher number of epochs since early stopping will prevent overfitting
                   validation_data=(x_test, y_test),
                   # TODO: Add the early stopping callback to the training process
                   callbacks=[________],
                   steps_per_epoch=len(x_train) // 32)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

# Get predictions
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate classification report
report = classification_report(y_test, y_pred_classes, target_names=categories)
print("\nClassification Report:")
print(report)

# Print per-class analysis
print("\nPer-class Analysis:")
print("------------------")
for i, category in enumerate(categories):
    # Find indices for this class
    class_indices = np.where(y_test == i)[0]
    class_pred = y_pred_classes[class_indices]
    class_true = y_test[class_indices]

    # Calculate accuracy for this class
    class_accuracy = np.mean(class_pred == class_true)

    # Count total samples for this class
    total_samples = len(class_indices)

    print(f"{category}: {class_accuracy:.2%} accuracy ({total_samples} samples)")

```

The error `AttributeError: module 'warnings' has no attribute 'warnings'` indicates that you are trying to access `warnings.warnings.filterwarnings`, but the `warnings` module itself does not have an attribute named `warnings`. You should directly call `filterwarnings` on the `warnings` module.

Here's the corrected line:

```python
import os
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="Your `PyDataset` class should call", category=UserWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import urllib.request
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# Data loading and preprocessing (already done for you)
categories = ['cat', 'house', 'apple']
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

os.makedirs('quickdraw_data', exist_ok=True)

for category in categories:
    file_path = f'quickdraw_data/{category}.npy'
    if not os.path.exists(file_path):
        print(f"Downloading {category}...")
        urllib.request.urlretrieve(base_url + category + '.npy', file_path)
    else:
        print(f"{category}.npy already exists.")

# Load and prepare data
data = []
labels = []
IMAGE_COUNT = 3000

for idx, cat in enumerate(categories):
    filepath = f'quickdraw_data/{cat}.npy'
    imgs = np.load(filepath)[:IMAGE_COUNT]
    if imgs.dtype != np.uint8:
        imgs = imgs.astype(np.uint8)
    data.append(imgs)
    labels.append(np.full(imgs.shape[0], idx))

# Combine all data and labels
data = np.concatenate(data, axis=0)
labels = np.concatenate(labels, axis=0)

# Shuffle data
indices = np.arange(len(data))
np.random.shuffle(indices)
data, labels = data[indices], labels[indices]

# Reshape and normalize
data = data.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(f"Training data shape: {x_train.shape}, Testing data shape: {x_test.shape}")

# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

# Fit the generator to the training data
datagen.fit(x_train)

def build_simple_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28,28,1)),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(categories), activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Build the model
model = build_simple_cnn()

# Print model summary
model.summary()

# Create an EarlyStopping callback that:
#  - Monitors validation accuracy
#  - Has a patience of 5 epochs
#  - Restores the best weights
#  - Provides verbose output
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model with early stopping
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                   epochs=1,  # We can set a higher number of epochs since early stopping will prevent overfitting
                   validation_data=(x_test, y_test),
                   # Add the early stopping callback to the training process
                   callbacks=[early_stopping],
                   steps_per_epoch=len(x_train) // 32)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

# Get predictions
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate classification report
report = classification_report(y_test, y_pred_classes, target_names=categories)
print("\nClassification Report:")
print(report)

# Print per-class analysis
print("\nPer-class Analysis:")
print("------------------")
for i, category in enumerate(categories):
    # Find indices for this class
    class_indices = np.where(y_test == i)[0]
    class_pred = y_pred_classes[class_indices]
    class_true = y_test[class_indices]

    # Calculate accuracy for this class
    class_accuracy = np.mean(class_pred == class_true)

    # Count total samples for this class
    total_samples = len(class_indices)

    print(f"{category}: {class_accuracy:.2%} accuracy ({total_samples} samples)")
```

Now that you've learned about techniques to improve model performance, simply analyze the final code which we will use for building the model for the upcoming course.

Here are the main steps of the code explained:

Data Loading and Preprocessing: Downloads the QuickDraw .npy files for each category, loads them, reshapes, and normalizes the images. The data is shuffled and split into training and test sets.

Data Augmentation: Uses ImageDataGenerator to apply random transformations (rotation, shift, zoom, shear) to the training images, helping the model generalize better.

Model Definition: Builds a CNN with three convolutional blocks, each followed by batch normalization and max pooling. A dense layer with batch normalization and a Dropout layer is added before the output. This helps prevent overfitting.

Early Stopping: Sets up an EarlyStopping callback to monitor validation accuracy and stop training when the model stops improving, restoring the best weights.

Label Preparation: Converts integer labels to one-hot encoded vectors for training with categorical cross-entropy loss.

Training: Trains the model using the augmented data, with early stopping enabled.

Evaluation: Evaluates the model on the test set and prints a classification report to show precision, recall, and F1-score for each class.

Saving the Model: Saves the trained model to disk for future use.

Note, running the code won't do anything, since the model training takes more than 30 minutes.

```python
import os
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="Your `PyDataset` class should call", category=UserWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import urllib.request
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# 1. Data loading and preprocessing
categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]
NUM_CLASSES = len(categories)
DATA_DIR = 'quickdraw_data'
os.makedirs(DATA_DIR, exist_ok=True)

for category in categories:
    file_path = os.path.join(DATA_DIR, f'{category}.npy')
    if not os.path.exists(file_path):
        print(f"Downloading {category}...")
        urllib.request.urlretrieve(
            f'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{category}.npy',
            file_path
        )
    else:
        print(f"{category}.npy already exists.")

data = []
labels = []
IMAGE_COUNT = 20000
IMAGE_SIZE = 28
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 1)

print("Loading and preparing data...")
for idx, cat in enumerate(categories):
    filepath = os.path.join(DATA_DIR, f'{cat}.npy')
    try:
        imgs = np.load(filepath)[:IMAGE_COUNT]
        if imgs.shape[1] != IMAGE_SIZE * IMAGE_SIZE:
            print(f"Warning: Unexpected data shape in {cat}.npy. Skipping.")
            continue
        if imgs.dtype != np.uint8:
            imgs = imgs.astype(np.uint8)
        imgs_reshaped = imgs.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
        data.append(imgs_reshaped)
        labels.append(np.full(imgs_reshaped.shape[0], idx))
    except Exception as e:
        print(f"Error loading or processing {cat}.npy: {e}")

if not data:
    print("Error: No data loaded. Exiting.")
    exit()

data = np.concatenate(data, axis=0)
labels = np.concatenate(labels, axis=0)
print(f"Total data shape: {data.shape}, Total labels shape: {labels.shape}")

# Shuffle and normalize data
indices = np.arange(len(data))
np.random.shuffle(indices)
data, labels = data[indices], labels[indices]
data = data.astype('float32') / 255.0

# Split data
x_train, x_test, y_train_indices, y_test_indices = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels
)
print(f"Training data shape: {x_train.shape}, Testing data shape: {x_test.shape}")

# 2. Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode='nearest'
)

# 3. Model definition with Dropout and Batch Normalization
def build_simple_cnn_with_dropout():
    model = tf.keras.models.Sequential(name="SketchCNN_with_Dropout")
    model.add(tf.keras.layers.Conv2D(32, 5, padding='same', activation='relu', input_shape=IMAGE_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    model.summary()
    return model

model = build_simple_cnn_with_dropout()

# 4. Early stopping callback
early_stopping_callback = EarlyStopping(
    patience=10,
    monitor='val_accuracy',
    restore_best_weights=True,
    verbose=1
)

# 5. Prepare labels for training
y_train_cat = tf.keras.utils.to_categorical(y_train_indices, num_classes=NUM_CLASSES)
y_test_cat = tf.keras.utils.to_categorical(y_test_indices, num_classes=NUM_CLASSES)

# 6. Training
BATCH_SIZE = 64
EPOCHS = 50
print("\n--- Starting Training ---")
history = model.fit(
    datagen.flow(x_train, y_train_cat, batch_size=BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=(x_test, y_test_cat),
    callbacks=[early_stopping_callback],
    verbose=1
)
print("\n--- Training Finished ---")

# 7. Evaluation
print("\n--- Evaluating Model on Test Set ---")
loss, accuracy = model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

print("\n--- Classification Report ---")
y_pred_probs = model.predict(x_test)
y_pred_indices = np.argmax(y_pred_probs, axis=1)
print(classification_report(y_test_indices, y_pred_indices, target_names=categories))

# 8. Save the model
MODEL_SAVE_PATH = 'models/drawing_cnn.keras'
model.save(MODEL_SAVE_PATH)
print(f"\nModel saved successfully to {MODEL_SAVE_PATH}!")


```

The provided code outlines a comprehensive approach to building and training a Convolutional Neural Network (CNN) for sketch recognition, incorporating several best practices for deep learning. Let's break down the analysis of this final code.

### Overall Structure and Purpose

The code's primary goal is to train a robust CNN capable of classifying hand-drawn sketches into 12 distinct categories from the QuickDraw dataset. It emphasizes techniques to prevent overfitting and ensure the model generalizes well to new, unseen sketches.

### Key Components and Their Significance

1.  **Data Loading and Preprocessing:**
    * **Increased Categories:** The model is now set up to classify 12 categories, which is a more realistic scenario compared to just 3.
    * **Larger `IMAGE_COUNT`:** Using `20000` images per category significantly expands the training data, which is crucial for deep learning models to learn complex patterns.
    * **Dynamic `NUM_CLASSES`:** `len(categories)` correctly sets the output layer size, making the model adaptable to changes in the number of categories.
    * **Robust Data Loading:** Includes error handling with `try-except` for file loading and a check for `imgs.shape[1]` to ensure data consistency, making the script more resilient.
    * **Stratified Splitting:** `stratify=labels` in `train_test_split` ensures that the proportion of each class is maintained in both the training and testing sets. This is vital for balanced datasets, especially with multiple classes, to avoid a situation where some classes are underrepresented in the test set.
    * **Normalization:** Images are normalized to `[0, 1]` by dividing by `255.0`, a standard practice that helps neural networks converge faster.

2.  **Data Augmentation (`ImageDataGenerator`):**
    * **Expanded Augmentation:** The `ImageDataGenerator` now uses wider ranges for `rotation_range`, `width_shift_range`, `height_shift_range`, `zoom_range`, and `shear_range` (e.g., 20 degrees for rotation, 0.15 for shifts/zooms/shears). This creates a much richer and more diverse training set from the existing images, making the model more robust to variations in drawing styles and minor distortions. This is a powerful technique to combat overfitting.

3.  **Model Definition (`build_simple_cnn_with_dropout`):**
    * **Deeper Architecture:** The model now features three convolutional blocks instead of two. More layers allow the model to learn more hierarchical and abstract features.
    * **`padding='same'`:** This ensures the output feature map has the same spatial dimensions as the input, which can be beneficial for preserving information at the borders of the images and making it easier to stack more layers.
    * **Batch Normalization (`BatchNormalization()`):** This is a critical addition after each convolutional block and before the final `Dense` layer. Batch Normalization helps:
        * Stabilize and speed up the training process.
        * Reduce internal covariate shift.
        * Act as a regularization technique, further reducing overfitting.
    * **`Dropout(0.5)`:** Applied to the `Dense` layer, this explicitly regularizes the network by randomly setting 50% of the inputs to zero during each training step. This prevents co-adaptation of neurons and makes the network more robust.
    * **Adam Optimizer with Learning Rate:** Explicitly defining `tf.keras.optimizers.Adam(learning_rate=0.001)` gives fine-grained control over the optimization process. A learning rate of 0.001 is a common and often effective starting point.
    * **`categorical_crossentropy` Loss:** This is the correct loss function for multi-class classification with one-hot encoded labels, which `tf.keras.utils.to_categorical` will provide.

4.  **Early Stopping Callback (`early_stopping_callback`):**
    * **`patience=10`:** This is a good value, allowing the model to continue training for 10 epochs after the `val_accuracy` stops improving before stopping. This accounts for minor fluctuations in validation performance.
    * **`restore_best_weights=True`:** Crucially, this ensures that once training stops, the model's weights are reverted to the epoch where `val_accuracy` was highest, providing the best performing model on unseen data.
    * **`verbose=1`:** Provides useful output during training about when early stopping is triggered.

5.  **Label Preparation (`to_categorical`):**
    * **One-Hot Encoding:** `tf.keras.utils.to_categorical` converts the integer labels (`y_train_indices`, `y_test_indices`) into one-hot encoded vectors (`y_train_cat`, `y_test_cat`). This is necessary because the `softmax` activation in the output layer and `categorical_crossentropy` loss expect this format for multi-class classification.

6.  **Training:**
    * **Increased Epochs (`EPOCHS = 50`):** While the previous example used `epochs=1` for quick demonstration, here `epochs=50` is set. This is a more realistic number for a deep learning model to converge, relying on `EarlyStopping` to prevent excessive training.
    * **Larger `BATCH_SIZE = 64`:** A batch size of 64 is common and provides a good balance between training speed and stable gradient updates.

7.  **Evaluation:**
    * **Standard Metrics:** `model.evaluate` provides overall loss and accuracy.
    * **Classification Report:** `sklearn.metrics.classification_report` is used to get detailed per-class metrics (precision, recall, F1-score), which are essential for understanding the model's performance on individual categories, especially if the dataset is imbalanced.

8.  **Saving the Model:**
    * **`model.save(MODEL_SAVE_PATH)`:** This is a vital step for production, allowing the trained model to be loaded later without retraining. The `.keras` extension is the recommended format for Keras models.

### Improvements over Previous Iterations

* **Scalability:** Increased number of categories and `IMAGE_COUNT` makes the solution more robust and applicable to larger datasets.
* **Robustness:** Incorporates `BatchNormalization` for more stable and faster training, and `Dropout` for stronger regularization.
* **Generalization:** Enhanced data augmentation and `EarlyStopping` are critical for building a model that performs well on unseen data, mitigating overfitting effectively.
* **Comprehensive Evaluation:** Detailed `classification_report` provides deeper insights into per-class performance, which is more informative than just overall accuracy.
* **Production Readiness:** Model saving is a crucial step towards deploying the model.

### Potential Considerations (Beyond the Scope of this Analysis)

* **Hyperparameter Tuning:** While good default values are used, optimizing learning rate, dropout rate, number of filters, and patience for early stopping could further improve performance.
* **Cross-validation:** For more robust evaluation, k-fold cross-validation could be used.
* **Class Imbalance Handling:** If the actual QuickDraw dataset for these 12 categories has significant class imbalance, techniques like class weighting or oversampling/undersampling might be considered, though `stratify` helps with the split.
* **Transfer Learning:** For very large and complex image datasets, using a pre-trained model (transfer learning) is often more efficient and effective.

In summary, this final code represents a well-structured and thoughtfully designed CNN for image classification, incorporating industry-standard techniques to achieve better performance and generalization.
